In [61]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [62]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [63]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [64]:
a = [13, 23,
     16, 20]
len(a)

4

In [65]:
sum(a[:2])

36

In [66]:
sum(a[2:])

36

## create new file for custom problem

In [90]:
import numpy as np

# Job durations
# durations = [13, 23, 13, 13, 13, 13, 13, 13, 23, 13, 10, 10, 10, 20, 25, 20, 10, 15, 10, 20]
durations = [13, 23,
     16, 20]

# Number of jobs
n = len(durations)

# Initialize QUBO matrix
Q = np.zeros((n, n))

# Fill the QUBO matrix
for i in range(n):
    for j in range(i, n):
        if i == j:
            Q[i, j] = durations[i] ** 2
        else:
            Q[i, j] = 2 * durations[i] * durations[j]

# Convert Q matrix to PyTorch tensor for consistency with previous code
Q = -Q
Q_tensor = torch.tensor(Q)  # Negate for minimization

# Display the QUBO matrix
print(Q_tensor)


tensor([[-169., -598., -416., -520.],
        [  -0., -529., -736., -920.],
        [  -0.,   -0., -256., -640.],
        [  -0.,   -0.,   -0., -400.]], dtype=torch.float64)


In [91]:
Q_tensor

tensor([[-169., -598., -416., -520.],
        [  -0., -529., -736., -920.],
        [  -0.,   -0., -256., -640.],
        [  -0.,   -0.,   -0., -400.]], dtype=torch.float64)

## save problem

In [92]:
# Generate the text file content
lines = []
lines.append("1\n")  # Number of problem instances

# Collect non-zero entries
non_zero_entries = []
for i in range(n):
    for j in range(i, n):
        if Q[i][j] != 0:
            non_zero_entries.append((i + 1, j + 1, int(Q[i][j])))

# Add the number of variables and number of non-zero entries
lines.append(f"{n} {len(non_zero_entries)}\n")

# Add the non-zero entries to lines
for entry in non_zero_entries:
    i, j, value = entry
    lines.append(f"{i} {j} {value}\n")

# Write to file
with open("../data/qubo/qubo_problem_small.txt", "w") as f:
    f.writelines(lines)

## load problem

In [93]:
problem = QUBO("../data/qubo/qubo_problem_small.txt", 0, device)

Instance has been loaded


In [94]:
problem

In [95]:
problem.Q

tensor([[-169., -598., -416., -520.],
        [-598., -529., -736., -920.],
        [-416., -736., -256., -640.],
        [-520., -920., -640., -400.]])

In [96]:
Q

array([[-169., -598., -416., -520.],
       [  -0., -529., -736., -920.],
       [  -0.,   -0., -256., -640.],
       [  -0.,   -0.,   -0., -400.]])

In [97]:
torch.count_nonzero(problem.Q, dim=0), torch.count_nonzero(problem.Q, dim=1)

(tensor([4, 4, 4, 4]), tensor([4, 4, 4, 4]))

In [98]:
torch.count_nonzero(Q_tensor, dim=0), torch.count_nonzero(Q_tensor, dim=1)

(tensor([1, 2, 3, 4]), tensor([4, 3, 2, 1]))

In [99]:
Q_tensor[1]

tensor([  -0., -529., -736., -920.], dtype=torch.float64)

In [100]:
problem.Q[1]

tensor([-598., -529., -736., -920.])